<a href="https://colab.research.google.com/github/Luisafe-ux/Business-Intelligence./blob/main/2_%C3%A1rboles_de_decisi%C3%B3n_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. se procede con la carga de las librerias de trabajo

In [4]:
#librerias científicas
import pandas as pd
import numpy as np

#librerias Arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#conectar al Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Descripción de los datos**

De acuerdo con la base de datos de solicitantes de credito esta posee un total de 5842 datos totales, divididos en 2959 prenegados y 2883 preAprobados. cada uno de los solicitantes de creditos estará descrito en función 4 variables:
*Edad: Indica el número de años de la persona.
*Ingresos:se muestra la cantidad de ingresos totales registrados en un mes por el solicitante.
*Egresos: Indica los gastos en los que incurre en  el mes una persona para cubrir sus necesidades básicas.
Monto (EAD): Es la cantidad solicitada por una persona a la FINTCH
Preapr: Indica la decsión del banco frente al otorgamiento de un crédito.

1.Se procede con la carga de los datos

In [7]:
nxl= '/content/drive/MyDrive/6. SolicitantesCrédito(USD).xlsx'
XDB=pd.read_excel(nxl, sheet_name=1) #Base AAA
XDB=XDB.dropna() #Eliminar filas vacias
XDB.head(100) #Mostrar 100 líneas del archivo

#Seleccionar las variables
XD=XDB [['Edad', 'Ingresos','Egresos','Monto (EAD)']]
yd=XDB[['PreApr']]

,Edad,Ingresos,Egresos,Monto (EAD)
0,38,1356.14400,1685.622667,625.562230
1,51,286.01600,364.462000,140.031984
2,37,517.46325,629.208889,284.564492
3,29,473.27000,551.616889,309.647588
4,42,750.09175,806.715778,500.663578
...,...,...,...,...
5837,48,1207.84800,753.801111,748.041791
5838,31,1472.77200,953.812889,870.793819
5839,38,773.01975,672.910667,594.947894
5840,43,635.50175,780.691556,305.580539


2. Se procede con la implementación del modelo

In [8]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

#Se procede con la gráfica}
from pydotplus import graph_from_dot_data #me crea la grafica
from sklearn.tree import export_graphviz #me exporta a PNG, JPEG, TIFF

ve=['Edad', 'Ingresos','Egresos','Monto (EAD)'] #Nombres para el árbol
dot_data=export_graphviz(mar, feature_names=ve) #Craer el grafico
graph=graph_from_dot_data(dot_data)
graph.write_png('arbol.png')


True

Los nodos puros (0). buscarlos en el arbol.
indice gini (0) o cercanos a gini


3. se procede con el pronocstico de los solicitantes que no poseeen PreApr

In [9]:
XDBO=pd.read_excel(nxl, sheet_name=0) #Base AAA
XDBO.head(100) #Mostrar 100 líneas del archivo

#Se crea la base de datos a pronosticar
XD0=XDBO [['Edad', 'Ingresos','Egresos','Monto (EAD)']]
yd0=XDBO[['PreApr']]

#Determinar las filas de los solicitantes que no poseen PreApr
filas=np.where(np.isnan(yd0))[0] #la segunda fila es un error

XD0=np.array(XD0) #Tabla operable
XD0=XD0[filas]

#Le calculamos la preaprobación a los solicitantes
y_pred=mar.predict(XD0)

#Se crea el informe en EXCEL
df=pd.DataFrame(np.column_stack((filas,XD0,y_pred)))
df.columns=['Solicitante','Edad','Ingresos','Egresos','Monto (EAD)','PreaApr']
df.to_excel("Evaluación Créditos.xlsx")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


4. Evaluar el comportamiento del modelo


In [10]:
ydp=mar.predict(XD)  #pronostico sobre la base de datos original
cm=confusion_matrix(yd,ydp)
print("La matriz de confusión es:\n",cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
print("La exactitud del modelo es:\n",Exactitud)

Sensibilidad=VP/(VP+FN)
print("La sensibilidad del modelo es:\n",Sensibilidad)

Especificidad=VN/(VN+FP)
print("La especificidad del modelo es:\n",Especificidad)

Precision=VP/(VP+FP)
print("La precisión del modelo es:\n",Precision)

TasaError=(FP+FN)/(VP+VN+FP+FN)
print("La tasa de error del modelo es:\n",TasaError)

PredNeg=VN/(VN+FN)
print("La tasa de falsos negativos del modelo es:\n",PredNeg)


La matriz de confusión es:
 [[2301  658]
 [ 644 2239]]
La exactitud del modelo es:
 0.7771311194796303
La sensibilidad del modelo es:
 0.7766215747485259
La especificidad del modelo es:
 0.7776275768840825
La precisión del modelo es:
 0.772868484639282
La tasa de error del modelo es:
 0.22286888052036974
La tasa de falsos negativos del modelo es:
 0.7813242784380305


**Análisis de resultados**

La matriz de confusión muestra una predominancia por los valores negativos (23021) y los valores de verdaderos positivos (2239). lo anterior hizo que el modelo alcanzara valores por encima del 75% para los indices exactitud, sensibilidad, especificidad,precisión lo que muestra claramente el buen funcionamiento del modelo frente al pronostico DE LA PREAPORBACIÓN. Con respecto al pronostico de nuevos solicitantes de crédito, la base de datos mostro un total de 2536 solicitantes sin decisión de crédito, se destacan un totak de NDA solicitudes preaprobadas y un total de NDN solicitudes prenegadas. Se destaca que el número de prenegados o preaprobados estuvo por encima del número de prenegados y preaprobados.
De acuerdo con los índices, el modelo muestra un mejor comportamiento frente a la especificidad (0.777) o clasificación de prenegados, a diferencia de la sensibilidad la cual estuvo por encima del 75% en promedio (0.776)
